### Conclusion from testing 
New function(s) for running experiments are doing the same as the old functions but the results are slightly different.<br> Likely caused by package updates. Even when I use the original sktime only environment. <br>
Results are comparable but slightly different <br> 
I tried re-running the old function in the same sktime_only environment and got slightly different results to the original <br> 
New function gives me same results as when I re-run the old function so I am fine with the new code <br>

In [5]:
# Combined environment may cause issues
'''ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.8.1 requires statsmodels<=0.12.1, but you have statsmodels 0.13.2 which is incompatible.
matrixprofile 1.1.10 requires protobuf==3.11.2, but you have protobuf 3.15.8 which is incompatible.'''

"ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.\nsktime 0.8.1 requires statsmodels<=0.12.1, but you have statsmodels 0.13.2 which is incompatible.\nmatrixprofile 1.1.10 requires protobuf==3.11.2, but you have protobuf 3.15.8 which is incompatible."

### Code from original script used to run expts (run_mv_tsc_expts_mrsql.py)

In [8]:
'''
o 1. PD vs HC,   2.(PD , PD+RBD) vs HC,   3.(PD + PD&RBD) vs HC vs RBD  multiclass, 4.PD vs PD+RBD vs RBD vs HC 
'''
core_path = '/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/'

#1 - This is to run the custom length expts
folders = {'N1': 'N1_customised_40_expt_files/' , 'N2' : 'N2_120_expt_files_v2/','N3' : 'N3_82_expt_files/', 'REM' :'REM_19_expt_files/'}
#2 -This is to run the expts on the full original length data without any dropping or cutting of the TS 
folders = {'N1': 'N1_full_expts/' , 'N2' : 'N2_full_expts/','N3' : 'N3_full_expts/', 'REM' :'REM_full_expts/'}
#3 - This is to run the wakeful eyes closed experiments
folders = {'EC1': 'EC_8_expts/' , 'EC2' : 'EC_full_expts/' }

#Folder = folders[sys.argv[1]] #User can input which experiments will be run
folder = folders['EC1'] #I'm choosing this because it's the TS & Therefore quickest hopefully?

expts = ['MrSQL_expt_1', 'MrSQL_expt_2' , 'MrSQL_expt_3' , 'MrSQL_expt_4']

power_band_cols = ['Delta', 'Theta', 'Alpha', 'Sigma', 'Beta', 'Gamma', 'TotalAbsPow']

regions = ['Prefrontal','Frontal', 'Left Frontal', 'Right Frontal', 'Central', 'Left Central', 
           'Right Central', 'Left Temporal','Right Temporal', 'Parietal', 'Left Parietal', 'Right Parietal', 'Occipital']

#Load the appropriate X (full set of features ) , y (full set of labels) , groups 
load_path = core_path + folder
X = pd.read_hdf(load_path + 'X.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'y_full.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'groups.h5', key='df', mode='r')  

In [5]:
# Code specific to this notebook starts below_____________________________________________________________________________

In [1]:
#Sktime imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.shapelet_based import MrSEQLClassifier
from sklearn.model_selection import GroupShuffleSplit

from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
import sys
from os.path import exists

import importlib

In [2]:
import constants
import custom_ts_length
# import eeg_stat_ts
import format_eeg_data
import run_expts

In [3]:
import sys
print(sys.executable)

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/bin/python


In [104]:
# .... I Think this needs to be run in the same sktime environment because results are slightly different and I think stats models impacts that!!

In [54]:
# Longest runtime = 4 clfs * 43 seconds per clf 
# Top end is 4 expts * that 
(4*43*4)/60

11.466666666666667

In [9]:
clfs = {'MrSQL_1' : MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) , 
    'MrSQL_2' : MrSEQLClassifier(seql_mode='clf', symrep=[ 'sfa' , 'sax']), 
    'MrSQL_3' : MrSEQLClassifier(seql_mode='clf', symrep=['sfa']), 
    'MrSQL_4':   MrSEQLClassifier(seql_mode='clf', symrep=['sax']) }

clfs = {'MrSQL_1' : MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) }

In [10]:
results_dict = {}
# for num in [1,2,3,4]:
for num in [1]:
    print('commencing for ' + str(num) + '...')

    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups,num)

    t1 = time.time()
    results_df = run_expts.run_mv_tsc(X_expt ,y_expt , groups_expt, clfs , return_df = True, random_states = [13 , 37])
    
    t2 = time.time()
    
    results_dict[num] = results_df
    
    print(t2-t1)

commencing for 1...
10.215959072113037


In [13]:
 run_expts.gen_subsets_dict()

{'Prefrontal': ['Delta_Prefrontal',
  'Theta_Prefrontal',
  'Alpha_Prefrontal',
  'Sigma_Prefrontal',
  'Beta_Prefrontal',
  'Gamma_Prefrontal',
  'TotalAbsPow_Prefrontal'],
 'Frontal': ['Delta_Frontal',
  'Theta_Frontal',
  'Alpha_Frontal',
  'Sigma_Frontal',
  'Beta_Frontal',
  'Gamma_Frontal',
  'TotalAbsPow_Frontal'],
 'Left Frontal': ['Delta_Left Frontal',
  'Theta_Left Frontal',
  'Alpha_Left Frontal',
  'Sigma_Left Frontal',
  'Beta_Left Frontal',
  'Gamma_Left Frontal',
  'TotalAbsPow_Left Frontal'],
 'Right Frontal': ['Delta_Right Frontal',
  'Theta_Right Frontal',
  'Alpha_Right Frontal',
  'Sigma_Right Frontal',
  'Beta_Right Frontal',
  'Gamma_Right Frontal',
  'TotalAbsPow_Right Frontal'],
 'Central': ['Delta_Central',
  'Theta_Central',
  'Alpha_Central',
  'Sigma_Central',
  'Beta_Central',
  'Gamma_Central',
  'TotalAbsPow_Central'],
 'Left Central': ['Delta_Left Central',
  'Theta_Left Central',
  'Alpha_Left Central',
  'Sigma_Left Central',
  'Beta_Left Central',
  '

In [11]:
results_df

,random_state,y_true,Prefrontal_MrSQL_1_y_preds,Frontal_MrSQL_1_y_preds,Left Frontal_MrSQL_1_y_preds,Right Frontal_MrSQL_1_y_preds,Central_MrSQL_1_y_preds,Left Central_MrSQL_1_y_preds,Right Central_MrSQL_1_y_preds,Left Temporal_MrSQL_1_y_preds,Right Temporal_MrSQL_1_y_preds,Parietal_MrSQL_1_y_preds,Left Parietal_MrSQL_1_y_preds,Right Parietal_MrSQL_1_y_preds,Occipital_MrSQL_1_y_preds
0,13,"[[0, 0, 0, 0, 1], [0, 0, 1, 1, 1], [0, 0, 0, 1...","[[0, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 0, 0, 0...","[[0, 0, 1, 1, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0...","[[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 0...","[[0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0...","[[1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0...","[[1, 0, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 1...","[[0, 0, 0, 1, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 0...","[[1, 1, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 0, 0, 1...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 1, 1, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0...","[[1, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 1, 1..."
1,37,"[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[1, 1, 0, 0, 1], [0, 1, 0, 1, 0], [1, 1, 0, 1...","[[0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 0...","[[0, 0, 1, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 1...","[[0, 0, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 1, 0...","[[0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 1...","[[1, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 0, 1...","[[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0...","[[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0...","[[1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 1...","[[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0...","[[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [1, 1, 0, 1...","[[1, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0..."


In [47]:
acc_std_results = {}
# for num in [1,2,3,4]:
for num in [1]:
    run_expts.generate_subset_acc_std(results_df, return_df = True)
    acc_std_results_df = run_expts.generate_subset_acc_std(results_dict[num], return_df = True)
    acc_std_results[num] = acc_std_results_df
acc_std_results[1]

,random_state,Prefrontal_MrSQL_1_acc,Prefrontal_MrSQL_1_std,Prefrontal_MrSQL_2_acc,Prefrontal_MrSQL_2_std,Prefrontal_MrSQL_3_acc,Prefrontal_MrSQL_3_std,Prefrontal_MrSQL_4_acc,Prefrontal_MrSQL_4_std,Frontal_MrSQL_1_acc,Frontal_MrSQL_1_std,Frontal_MrSQL_2_acc,Frontal_MrSQL_2_std,Frontal_MrSQL_3_acc,Frontal_MrSQL_3_std,Frontal_MrSQL_4_acc,Frontal_MrSQL_4_std,Left Frontal_MrSQL_1_acc,Left Frontal_MrSQL_1_std,Left Frontal_MrSQL_2_acc,Left Frontal_MrSQL_2_std,Left Frontal_MrSQL_3_acc,Left Frontal_MrSQL_3_std,Left Frontal_MrSQL_4_acc,Left Frontal_MrSQL_4_std,Right Frontal_MrSQL_1_acc,Right Frontal_MrSQL_1_std,Right Frontal_MrSQL_2_acc,Right Frontal_MrSQL_2_std,Right Frontal_MrSQL_3_acc,Right Frontal_MrSQL_3_std,Right Frontal_MrSQL_4_acc,Right Frontal_MrSQL_4_std,Central_MrSQL_1_acc,Central_MrSQL_1_std,Central_MrSQL_2_acc,Central_MrSQL_2_std,Central_MrSQL_3_acc,Central_MrSQL_3_std,Central_MrSQL_4_acc,Central_MrSQL_4_std,Left Central_MrSQL_1_acc,Left Central_MrSQL_1_std,Left Central_MrSQL_2_acc,Left Central_MrSQL_2_std,Left Central_MrSQL_3_acc,Left Central_MrSQL_3_std,Left Central_MrSQL_4_acc,Left Central_MrSQL_4_std,Right Central_MrSQL_1_acc,Right Central_MrSQL_1_std,Right Central_MrSQL_2_acc,Right Central_MrSQL_2_std,Right Central_MrSQL_3_acc,Right Central_MrSQL_3_std,Right Central_MrSQL_4_acc,Right Central_MrSQL_4_std,Left Temporal_MrSQL_1_acc,Left Temporal_MrSQL_1_std,Left Temporal_MrSQL_2_acc,Left Temporal_MrSQL_2_std,Left Temporal_MrSQL_3_acc,Left Temporal_MrSQL_3_std,Left Temporal_MrSQL_4_acc,Left Temporal_MrSQL_4_std,Right Temporal_MrSQL_1_acc,Right Temporal_MrSQL_1_std,Right Temporal_MrSQL_2_acc,Right Temporal_MrSQL_2_std,Right Temporal_MrSQL_3_acc,Right Temporal_MrSQL_3_std,Right Temporal_MrSQL_4_acc,Right Temporal_MrSQL_4_std,Parietal_MrSQL_1_acc,Parietal_MrSQL_1_std,Parietal_MrSQL_2_acc,Parietal_MrSQL_2_std,Parietal_MrSQL_3_acc,Parietal_MrSQL_3_std,Parietal_MrSQL_4_acc,Parietal_MrSQL_4_std,Left Parietal_MrSQL_1_acc,Left Parietal_MrSQL_1_std,Left Parietal_MrSQL_2_acc,Left Parietal_MrSQL_2_std,Left Parietal_MrSQL_3_acc,Left Parietal_MrSQL_3_std,Left Parietal_MrSQL_4_acc,Left Parietal_MrSQL_4_std,Right Parietal_MrSQL_1_acc,Right Parietal_MrSQL_1_std,Right Parietal_MrSQL_2_acc,Right Parietal_MrSQL_2_std,Right Parietal_MrSQL_3_acc,Right Parietal_MrSQL_3_std,Right Parietal_MrSQL_4_acc,Right Parietal_MrSQL_4_std,Occipital_MrSQL_1_acc,Occipital_MrSQL_1_std,Occipital_MrSQL_2_acc,Occipital_MrSQL_2_std,Occipital_MrSQL_3_acc,Occipital_MrSQL_3_std,Occipital_MrSQL_4_acc,Occipital_MrSQL_4_std
0,13,0.80,0.126491,0.76,0.149666,0.60,0.178885,0.80,0.126491,0.32,0.097980,0.68,0.097980,0.64,0.149666,0.72,0.09798,0.64,0.080000,0.52,0.097980,0.56,0.233238,0.64,0.080000,0.40,0.126491,0.6,0.252982,0.64,0.233238,0.60,0.178885,0.48,0.09798,0.44,0.149666,0.48,0.160000,0.36,0.149666,0.44,0.149666,0.64,0.149666,0.68,0.203961,0.56,0.195959,0.60,0.126491,0.52,0.097980,0.40,0.178885,0.36,0.080000,0.52,0.16000,0.48,0.097980,0.48,0.09798,0.36,0.195959,0.52,0.203961,0.72,0.09798,0.52,0.09798,0.76,0.233238,0.60,0.000000,0.68,0.097980,0.84,0.149666,0.40,0.126491,0.64,0.149666,0.56,0.233238,0.52,0.097980,0.60,0.178885,0.56,0.149666,0.60,0.126491,0.24,0.149666,0.64,0.149666,0.76,0.149666,0.60,0.126491,0.52,0.097980,0.52,0.271293
1,37,0.92,0.097980,0.92,0.097980,0.84,0.149666,0.92,0.097980,0.32,0.203961,0.68,0.203961,0.60,0.357771,0.76,0.08000,0.64,0.195959,0.64,0.149666,0.56,0.080000,0.80,0.126491,0.64,0.233238,0.8,0.126491,0.80,0.178885,0.72,0.097980,0.68,0.09798,0.76,0.233238,0.64,0.149666,0.52,0.097980,0.40,0.334664,0.88,0.097980,0.72,0.203961,0.64,0.149666,0.52,0.240000,0.72,0.203961,0.52,0.160000,0.44,0.149666,0.52,0.09798,0.60,0.282843,0.68,0.16000,0.52,0.271293,0.52,0.097980,0.84,0.08000,0.72,0.09798,0.76,0.149666,0.48,0.203961,0.80,0.178885,0.92,0.097980,0.56,0.233238,0.72,0.203961,0.80,0.126491,0.60,0.126491,0.84,0.149666,0.40,0.252982,0.72,0.203961,0.60,0.282843,0.80,0.252982,0.60,0.178885,0.84,0.080000,0.76,0.233238,0.76,0.149666


In [68]:
###Load the original results to compare

In [48]:
root_folder = '/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/EC_8_expts/'
sub_folders = ['MrSQL_expt_1/', 'MrSQL_expt_2/', 'MrSQL_expt_3/', 'MrSQL_expt_4/']
files = ['int_df.h5','raw_clfs_results.h5','regional_clf_acc_std_results.h5', ' raw_ensemble_predictions.h5','processed_results_full_ensemble_df.h5']

In [49]:
results_dict_original = {}
acc_std_results_original = {}
# for num in [1,2,3,4]:
for num in [1]:
    folder = 'MrSQL_expt_' + str(num)
    results_dict_original[num] = pd.read_hdf(root_folder+folder+'/raw_clfs_results.h5')
    acc_std_results_original[num] = pd.read_hdf(root_folder+folder+'/regional_clf_acc_std_results.h5')

In [13]:
###Save the dataframes to compare so that I can reload easily after exiting this notebook

In [14]:
# This file was created and dumped using the sktime_and_rbd environment, giving different results so I am going to switch 
# To sktime_only environment and re-do the check
# lst_of_stuffs = [results_dict_original, acc_std_results_original, results_dict , acc_std_results]
# joblib.dump(lst_of_stuffs, 'list_of_results_dicts.pkl')
# list_of_results_dicts = joblib.load('list_of_results_dicts.pkl')

### Do the comparison

In [50]:
num = 1
acc_std_results_original[num].equals(acc_std_results[num])

False

In [51]:
acc_std_results[1].equals(acc_std_results_original[1])

False

In [52]:
df_new = acc_std_results[1]

In [53]:
df_orig = acc_std_results_original[1]

In [54]:
results_dict[1]['Prefrontal_MrSQL_1_y_preds'][1]

[array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1]),
 array([0, 0, 0, 1, 1]),
 array([0, 0, 0, 0, 1])]

In [55]:
results_dict_original[1]['Prefrontal_MrSQL_1_y_preds'][1]

[array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1]),
 array([0, 0, 0, 1, 1]),
 array([0, 0, 0, 0, 1])]

In [31]:
df_new

,random_state,Prefrontal_MrSQL_1_acc,Prefrontal_MrSQL_1_std,Prefrontal_MrSQL_2_acc,Prefrontal_MrSQL_2_std,Prefrontal_MrSQL_3_acc,Prefrontal_MrSQL_3_std,Prefrontal_MrSQL_4_acc,Prefrontal_MrSQL_4_std,Frontal_MrSQL_1_acc,Frontal_MrSQL_1_std,Frontal_MrSQL_2_acc,Frontal_MrSQL_2_std,Frontal_MrSQL_3_acc,Frontal_MrSQL_3_std,Frontal_MrSQL_4_acc,Frontal_MrSQL_4_std,Left Frontal_MrSQL_1_acc,Left Frontal_MrSQL_1_std,Left Frontal_MrSQL_2_acc,Left Frontal_MrSQL_2_std,Left Frontal_MrSQL_3_acc,Left Frontal_MrSQL_3_std,Left Frontal_MrSQL_4_acc,Left Frontal_MrSQL_4_std,Right Frontal_MrSQL_1_acc,Right Frontal_MrSQL_1_std,Right Frontal_MrSQL_2_acc,Right Frontal_MrSQL_2_std,Right Frontal_MrSQL_3_acc,Right Frontal_MrSQL_3_std,Right Frontal_MrSQL_4_acc,Right Frontal_MrSQL_4_std,Central_MrSQL_1_acc,Central_MrSQL_1_std,Central_MrSQL_2_acc,Central_MrSQL_2_std,Central_MrSQL_3_acc,Central_MrSQL_3_std,Central_MrSQL_4_acc,Central_MrSQL_4_std,Left Central_MrSQL_1_acc,Left Central_MrSQL_1_std,Left Central_MrSQL_2_acc,Left Central_MrSQL_2_std,Left Central_MrSQL_3_acc,Left Central_MrSQL_3_std,Left Central_MrSQL_4_acc,Left Central_MrSQL_4_std,Right Central_MrSQL_1_acc,Right Central_MrSQL_1_std,Right Central_MrSQL_2_acc,Right Central_MrSQL_2_std,Right Central_MrSQL_3_acc,Right Central_MrSQL_3_std,Right Central_MrSQL_4_acc,Right Central_MrSQL_4_std,Left Temporal_MrSQL_1_acc,Left Temporal_MrSQL_1_std,Left Temporal_MrSQL_2_acc,Left Temporal_MrSQL_2_std,Left Temporal_MrSQL_3_acc,Left Temporal_MrSQL_3_std,Left Temporal_MrSQL_4_acc,Left Temporal_MrSQL_4_std,Right Temporal_MrSQL_1_acc,Right Temporal_MrSQL_1_std,Right Temporal_MrSQL_2_acc,Right Temporal_MrSQL_2_std,Right Temporal_MrSQL_3_acc,Right Temporal_MrSQL_3_std,Right Temporal_MrSQL_4_acc,Right Temporal_MrSQL_4_std,Parietal_MrSQL_1_acc,Parietal_MrSQL_1_std,Parietal_MrSQL_2_acc,Parietal_MrSQL_2_std,Parietal_MrSQL_3_acc,Parietal_MrSQL_3_std,Parietal_MrSQL_4_acc,Parietal_MrSQL_4_std,Left Parietal_MrSQL_1_acc,Left Parietal_MrSQL_1_std,Left Parietal_MrSQL_2_acc,Left Parietal_MrSQL_2_std,Left Parietal_MrSQL_3_acc,Left Parietal_MrSQL_3_std,Left Parietal_MrSQL_4_acc,Left Parietal_MrSQL_4_std,Right Parietal_MrSQL_1_acc,Right Parietal_MrSQL_1_std,Right Parietal_MrSQL_2_acc,Right Parietal_MrSQL_2_std,Right Parietal_MrSQL_3_acc,Right Parietal_MrSQL_3_std,Right Parietal_MrSQL_4_acc,Right Parietal_MrSQL_4_std,Occipital_MrSQL_1_acc,Occipital_MrSQL_1_std,Occipital_MrSQL_2_acc,Occipital_MrSQL_2_std,Occipital_MrSQL_3_acc,Occipital_MrSQL_3_std,Occipital_MrSQL_4_acc,Occipital_MrSQL_4_std
0,13,0.80,0.126491,0.76,0.149666,0.60,0.178885,0.80,0.126491,0.32,0.097980,0.68,0.097980,0.64,0.149666,0.72,0.09798,0.64,0.080000,0.52,0.097980,0.56,0.233238,0.64,0.080000,0.40,0.126491,0.6,0.252982,0.64,0.233238,0.60,0.178885,0.48,0.09798,0.44,0.149666,0.48,0.160000,0.36,0.149666,0.44,0.149666,0.64,0.149666,0.68,0.203961,0.56,0.195959,0.60,0.126491,0.52,0.097980,0.40,0.178885,0.36,0.080000,0.52,0.16000,0.48,0.097980,0.48,0.09798,0.36,0.195959,0.52,0.203961,0.72,0.09798,0.52,0.09798,0.76,0.233238,0.60,0.000000,0.68,0.097980,0.84,0.149666,0.40,0.126491,0.64,0.149666,0.56,0.233238,0.52,0.097980,0.60,0.178885,0.56,0.149666,0.60,0.126491,0.24,0.149666,0.64,0.149666,0.76,0.149666,0.60,0.126491,0.52,0.097980,0.52,0.271293
1,37,0.92,0.097980,0.92,0.097980,0.84,0.149666,0.92,0.097980,0.32,0.203961,0.68,0.203961,0.60,0.357771,0.76,0.08000,0.64,0.195959,0.64,0.149666,0.56,0.080000,0.80,0.126491,0.64,0.233238,0.8,0.126491,0.80,0.178885,0.72,0.097980,0.68,0.09798,0.76,0.233238,0.64,0.149666,0.52,0.097980,0.40,0.334664,0.88,0.097980,0.72,0.203961,0.64,0.149666,0.52,0.240000,0.72,0.203961,0.52,0.160000,0.44,0.149666,0.52,0.09798,0.60,0.282843,0.68,0.16000,0.52,0.271293,0.52,0.097980,0.84,0.08000,0.72,0.09798,0.76,0.149666,0.48,0.203961,0.80,0.178885,0.92,0.097980,0.56,0.233238,0.72,0.203961,0.80,0.126491,0.60,0.126491,0.84,0.149666,0.40,0.252982,0.72,0.203961,0.60,0.282843,0.80,0.252982,0.60,0.178885,0.84,0.080000,0.76,0.233238,0.76,0.149666


In [32]:
df_orig

,random_state,Prefrontal_MrSQL_1_acc,Prefrontal_MrSQL_1_std,Prefrontal_MrSQL_2_acc,Prefrontal_MrSQL_2_std,Prefrontal_MrSQL_3_acc,Prefrontal_MrSQL_3_std,Prefrontal_MrSQL_4_acc,Prefrontal_MrSQL_4_std,Frontal_MrSQL_1_acc,Frontal_MrSQL_1_std,Frontal_MrSQL_2_acc,Frontal_MrSQL_2_std,Frontal_MrSQL_3_acc,Frontal_MrSQL_3_std,Frontal_MrSQL_4_acc,Frontal_MrSQL_4_std,Left Frontal_MrSQL_1_acc,Left Frontal_MrSQL_1_std,Left Frontal_MrSQL_2_acc,Left Frontal_MrSQL_2_std,Left Frontal_MrSQL_3_acc,Left Frontal_MrSQL_3_std,Left Frontal_MrSQL_4_acc,Left Frontal_MrSQL_4_std,Right Frontal_MrSQL_1_acc,Right Frontal_MrSQL_1_std,Right Frontal_MrSQL_2_acc,Right Frontal_MrSQL_2_std,Right Frontal_MrSQL_3_acc,Right Frontal_MrSQL_3_std,Right Frontal_MrSQL_4_acc,Right Frontal_MrSQL_4_std,Central_MrSQL_1_acc,Central_MrSQL_1_std,Central_MrSQL_2_acc,Central_MrSQL_2_std,Central_MrSQL_3_acc,Central_MrSQL_3_std,Central_MrSQL_4_acc,Central_MrSQL_4_std,Left Central_MrSQL_1_acc,Left Central_MrSQL_1_std,Left Central_MrSQL_2_acc,Left Central_MrSQL_2_std,Left Central_MrSQL_3_acc,Left Central_MrSQL_3_std,Left Central_MrSQL_4_acc,Left Central_MrSQL_4_std,Right Central_MrSQL_1_acc,Right Central_MrSQL_1_std,Right Central_MrSQL_2_acc,Right Central_MrSQL_2_std,Right Central_MrSQL_3_acc,Right Central_MrSQL_3_std,Right Central_MrSQL_4_acc,Right Central_MrSQL_4_std,Left Temporal_MrSQL_1_acc,Left Temporal_MrSQL_1_std,Left Temporal_MrSQL_2_acc,Left Temporal_MrSQL_2_std,Left Temporal_MrSQL_3_acc,Left Temporal_MrSQL_3_std,Left Temporal_MrSQL_4_acc,Left Temporal_MrSQL_4_std,Right Temporal_MrSQL_1_acc,Right Temporal_MrSQL_1_std,Right Temporal_MrSQL_2_acc,Right Temporal_MrSQL_2_std,Right Temporal_MrSQL_3_acc,Right Temporal_MrSQL_3_std,Right Temporal_MrSQL_4_acc,Right Temporal_MrSQL_4_std,Parietal_MrSQL_1_acc,Parietal_MrSQL_1_std,Parietal_MrSQL_2_acc,Parietal_MrSQL_2_std,Parietal_MrSQL_3_acc,Parietal_MrSQL_3_std,Parietal_MrSQL_4_acc,Parietal_MrSQL_4_std,Left Parietal_MrSQL_1_acc,Left Parietal_MrSQL_1_std,Left Parietal_MrSQL_2_acc,Left Parietal_MrSQL_2_std,Left Parietal_MrSQL_3_acc,Left Parietal_MrSQL_3_std,Left Parietal_MrSQL_4_acc,Left Parietal_MrSQL_4_std,Right Parietal_MrSQL_1_acc,Right Parietal_MrSQL_1_std,Right Parietal_MrSQL_2_acc,Right Parietal_MrSQL_2_std,Right Parietal_MrSQL_3_acc,Right Parietal_MrSQL_3_std,Right Parietal_MrSQL_4_acc,Right Parietal_MrSQL_4_std,Occipital_MrSQL_1_acc,Occipital_MrSQL_1_std,Occipital_MrSQL_2_acc,Occipital_MrSQL_2_std,Occipital_MrSQL_3_acc,Occipital_MrSQL_3_std,Occipital_MrSQL_4_acc,Occipital_MrSQL_4_std
0,13,0.80,0.126491,0.76,0.149666,0.60,0.178885,0.80,0.126491,0.32,0.097980,0.68,0.097980,0.64,0.149666,0.72,0.09798,0.6,0.126491,0.52,0.097980,0.56,0.233238,0.64,0.080000,0.36,0.080000,0.6,0.252982,0.64,0.233238,0.60,0.178885,0.48,0.09798,0.44,0.149666,0.48,0.160000,0.36,0.149666,0.4,0.219089,0.64,0.149666,0.68,0.203961,0.56,0.195959,0.60,0.126491,0.52,0.097980,0.40,0.178885,0.36,0.080000,0.56,0.149666,0.48,0.097980,0.48,0.09798,0.36,0.195959,0.44,0.195959,0.72,0.09798,0.52,0.09798,0.76,0.233238,0.60,0.000000,0.68,0.097980,0.84,0.149666,0.40,0.126491,0.56,0.233238,0.56,0.233238,0.52,0.097980,0.60,0.178885,0.56,0.149666,0.60,0.126491,0.24,0.149666,0.64,0.149666,0.76,0.149666,0.60,0.126491,0.52,0.097980,0.52,0.271293
1,37,0.92,0.097980,0.92,0.097980,0.84,0.149666,0.92,0.097980,0.40,0.282843,0.68,0.203961,0.60,0.357771,0.76,0.08000,0.6,0.178885,0.64,0.149666,0.56,0.080000,0.80,0.126491,0.64,0.233238,0.8,0.126491,0.80,0.178885,0.72,0.097980,0.68,0.09798,0.76,0.233238,0.64,0.149666,0.52,0.097980,0.4,0.334664,0.88,0.097980,0.72,0.203961,0.64,0.149666,0.56,0.293939,0.72,0.203961,0.52,0.160000,0.44,0.149666,0.52,0.097980,0.60,0.282843,0.68,0.16000,0.52,0.271293,0.52,0.097980,0.84,0.08000,0.72,0.09798,0.76,0.149666,0.48,0.203961,0.80,0.178885,0.92,0.097980,0.56,0.233238,0.72,0.203961,0.80,0.126491,0.60,0.126491,0.84,0.149666,0.40,0.252982,0.72,0.203961,0.60,0.282843,0.80,0.252982,0.60,0.178885,0.84,0.080000,0.76,0.233238,0.76,0.149666


In [43]:
true_f_df = (df_new - df_orig) == 0
true_f_df[[ col for col in list(true_f_df.columns) if 'acc' in col]]

,Prefrontal_MrSQL_1_acc,Prefrontal_MrSQL_2_acc,Prefrontal_MrSQL_3_acc,Prefrontal_MrSQL_4_acc,Frontal_MrSQL_1_acc,Frontal_MrSQL_2_acc,Frontal_MrSQL_3_acc,Frontal_MrSQL_4_acc,Left Frontal_MrSQL_1_acc,Left Frontal_MrSQL_2_acc,Left Frontal_MrSQL_3_acc,Left Frontal_MrSQL_4_acc,Right Frontal_MrSQL_1_acc,Right Frontal_MrSQL_2_acc,Right Frontal_MrSQL_3_acc,Right Frontal_MrSQL_4_acc,Central_MrSQL_1_acc,Central_MrSQL_2_acc,Central_MrSQL_3_acc,Central_MrSQL_4_acc,Left Central_MrSQL_1_acc,Left Central_MrSQL_2_acc,Left Central_MrSQL_3_acc,Left Central_MrSQL_4_acc,Right Central_MrSQL_1_acc,Right Central_MrSQL_2_acc,Right Central_MrSQL_3_acc,Right Central_MrSQL_4_acc,Left Temporal_MrSQL_1_acc,Left Temporal_MrSQL_2_acc,Left Temporal_MrSQL_3_acc,Left Temporal_MrSQL_4_acc,Right Temporal_MrSQL_1_acc,Right Temporal_MrSQL_2_acc,Right Temporal_MrSQL_3_acc,Right Temporal_MrSQL_4_acc,Parietal_MrSQL_1_acc,Parietal_MrSQL_2_acc,Parietal_MrSQL_3_acc,Parietal_MrSQL_4_acc,Left Parietal_MrSQL_1_acc,Left Parietal_MrSQL_2_acc,Left Parietal_MrSQL_3_acc,Left Parietal_MrSQL_4_acc,Right Parietal_MrSQL_1_acc,Right Parietal_MrSQL_2_acc,Right Parietal_MrSQL_3_acc,Right Parietal_MrSQL_4_acc,Occipital_MrSQL_1_acc,Occipital_MrSQL_2_acc,Occipital_MrSQL_3_acc,Occipital_MrSQL_4_acc
0,True,True,True,True,True,True,True,True,False,True,True,True,False,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,False,True,True,True,False,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,False,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [56]:
(df_new - df_orig)

,random_state,Prefrontal_MrSQL_1_acc,Prefrontal_MrSQL_1_std,Prefrontal_MrSQL_2_acc,Prefrontal_MrSQL_2_std,Prefrontal_MrSQL_3_acc,Prefrontal_MrSQL_3_std,Prefrontal_MrSQL_4_acc,Prefrontal_MrSQL_4_std,Frontal_MrSQL_1_acc,Frontal_MrSQL_1_std,Frontal_MrSQL_2_acc,Frontal_MrSQL_2_std,Frontal_MrSQL_3_acc,Frontal_MrSQL_3_std,Frontal_MrSQL_4_acc,Frontal_MrSQL_4_std,Left Frontal_MrSQL_1_acc,Left Frontal_MrSQL_1_std,Left Frontal_MrSQL_2_acc,Left Frontal_MrSQL_2_std,Left Frontal_MrSQL_3_acc,Left Frontal_MrSQL_3_std,Left Frontal_MrSQL_4_acc,Left Frontal_MrSQL_4_std,Right Frontal_MrSQL_1_acc,Right Frontal_MrSQL_1_std,Right Frontal_MrSQL_2_acc,Right Frontal_MrSQL_2_std,Right Frontal_MrSQL_3_acc,Right Frontal_MrSQL_3_std,Right Frontal_MrSQL_4_acc,Right Frontal_MrSQL_4_std,Central_MrSQL_1_acc,Central_MrSQL_1_std,Central_MrSQL_2_acc,Central_MrSQL_2_std,Central_MrSQL_3_acc,Central_MrSQL_3_std,Central_MrSQL_4_acc,Central_MrSQL_4_std,Left Central_MrSQL_1_acc,Left Central_MrSQL_1_std,Left Central_MrSQL_2_acc,Left Central_MrSQL_2_std,Left Central_MrSQL_3_acc,Left Central_MrSQL_3_std,Left Central_MrSQL_4_acc,Left Central_MrSQL_4_std,Right Central_MrSQL_1_acc,Right Central_MrSQL_1_std,Right Central_MrSQL_2_acc,Right Central_MrSQL_2_std,Right Central_MrSQL_3_acc,Right Central_MrSQL_3_std,Right Central_MrSQL_4_acc,Right Central_MrSQL_4_std,Left Temporal_MrSQL_1_acc,Left Temporal_MrSQL_1_std,Left Temporal_MrSQL_2_acc,Left Temporal_MrSQL_2_std,Left Temporal_MrSQL_3_acc,Left Temporal_MrSQL_3_std,Left Temporal_MrSQL_4_acc,Left Temporal_MrSQL_4_std,Right Temporal_MrSQL_1_acc,Right Temporal_MrSQL_1_std,Right Temporal_MrSQL_2_acc,Right Temporal_MrSQL_2_std,Right Temporal_MrSQL_3_acc,Right Temporal_MrSQL_3_std,Right Temporal_MrSQL_4_acc,Right Temporal_MrSQL_4_std,Parietal_MrSQL_1_acc,Parietal_MrSQL_1_std,Parietal_MrSQL_2_acc,Parietal_MrSQL_2_std,Parietal_MrSQL_3_acc,Parietal_MrSQL_3_std,Parietal_MrSQL_4_acc,Parietal_MrSQL_4_std,Left Parietal_MrSQL_1_acc,Left Parietal_MrSQL_1_std,Left Parietal_MrSQL_2_acc,Left Parietal_MrSQL_2_std,Left Parietal_MrSQL_3_acc,Left Parietal_MrSQL_3_std,Left Parietal_MrSQL_4_acc,Left Parietal_MrSQL_4_std,Right Parietal_MrSQL_1_acc,Right Parietal_MrSQL_1_std,Right Parietal_MrSQL_2_acc,Right Parietal_MrSQL_2_std,Right Parietal_MrSQL_3_acc,Right Parietal_MrSQL_3_std,Right Parietal_MrSQL_4_acc,Right Parietal_MrSQL_4_std,Occipital_MrSQL_1_acc,Occipital_MrSQL_1_std,Occipital_MrSQL_2_acc,Occipital_MrSQL_2_std,Occipital_MrSQL_3_acc,Occipital_MrSQL_3_std,Occipital_MrSQL_4_acc,Occipital_MrSQL_4_std
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.04,-0.046491,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.046491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,-0.069423,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.04,0.010334,0.0,0.0,0.0,0.0,0.0,0.0,0.08,0.008002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08,-0.083572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.08,-0.078882,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.017074,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.04,-0.053939,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
-9.040616204608165e-02

-0.09040616204608165

In [28]:
pd.options.display.max_columns = 250 #Changes the number of columns diplayed (default is 20)
pd.options.display.max_rows = 250 #Changes the number of rows diplayed (default is 60)
pd.options.display.max_colwidth = 250 #Changes the number of characters in a cell so that the contents don't get truncated (default is 50)

### Run the old function to see if the function is generating different results somehow in the current env

In [57]:
save_path = '/export/sphere/ebirah/ko20929/RBD_using_custom_package/'

clfs = {'MrSQL_1' : MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) , 
    'MrSQL_2' : MrSEQLClassifier(seql_mode='clf', symrep=[ 'sfa' , 'sax']), 
    'MrSQL_3' : MrSEQLClassifier(seql_mode='clf', symrep=['sfa']), 
    'MrSQL_4':   MrSEQLClassifier(seql_mode='clf', symrep=['sax']) }

results_dict = {}
# for num in [1,2,3,4]:
for num in [1]:
    print('commencing for ' + str(num) + '...')

    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups,num)

    t1 = time.time()
    run_expts.run_mv_tsc_old(X_expt ,y_expt , groups_expt, clfs , save_path = save_path, random_states = [13 , 37])
    raw_results = pd.read_hdf(save_path + 'int_df.h5'  , key='df', mode='r')
    
    t2 = time.time()
    
    results_dict[num] = results_df
    
    print(t2-t1)

commencing for 1...


/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['y_true', 'Prefrontal_MrSQL_1_y_preds', 'Prefrontal_MrSQL_2_y_preds',
       'Prefrontal_MrSQL_3_y_preds', 'Prefrontal_MrSQL_4_y_preds',
       'Frontal_MrSQL_1_y_preds', 'Frontal_MrSQL_2_y_preds',
       'Frontal_MrSQL_3_y_preds', 'Frontal_MrSQL_4_y_preds',
       'Left Frontal_MrSQL_1_y_preds', 'Left Frontal_MrSQL_2_y_preds',
       'Left Frontal_MrSQL_3_y_preds', 'Left Frontal_MrSQL_4_y_preds',
       'Right Frontal_MrSQL_1_y_preds', 'Right Frontal_MrSQL_2_y_preds',
       'Right Frontal_MrSQL_3_y_preds', 'Right Frontal_MrSQL_4_y_preds',
       'Central_MrSQL_1_y_preds', 'Central_MrSQL_2_y_preds',
       'Central_MrSQL_3_y_preds', 'Central_MrSQL_4_y_preds',
       'Left Central

31.53182339668274


/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['y_true', 'Prefrontal_MrSQL_1_y_preds', 'Prefrontal_MrSQL_2_y_preds',
       'Prefrontal_MrSQL_3_y_preds', 'Prefrontal_MrSQL_4_y_preds',
       'Frontal_MrSQL_1_y_preds', 'Frontal_MrSQL_2_y_preds',
       'Frontal_MrSQL_3_y_preds', 'Frontal_MrSQL_4_y_preds',
       'Left Frontal_MrSQL_1_y_preds', 'Left Frontal_MrSQL_2_y_preds',
       'Left Frontal_MrSQL_3_y_preds', 'Left Frontal_MrSQL_4_y_preds',
       'Right Frontal_MrSQL_1_y_preds', 'Right Frontal_MrSQL_2_y_preds',
       'Right Frontal_MrSQL_3_y_preds', 'Right Frontal_MrSQL_4_y_preds',
       'Central_MrSQL_1_y_preds', 'Central_MrSQL_2_y_preds',
       'Central_MrSQL_3_y_preds', 'Central_MrSQL_4_y_preds',
       'Left Central

In [58]:
new_2_df = run_expts.generate_subset_acc_std(results_dict[num], return_df = True)

In [60]:
new_2_df.equals(df_new)

True